<a href="https://colab.research.google.com/github/revans011/Minutes_of_Motion/blob/main/Canine_Minutes_in_Motion_Algorithm_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Minutes in Motion Algorithm

##This notebook demonstrates how to use IMU data to count the number of minutes dogs move during a clinical trial

##Introduction
Inertial measurement units (IMUs) contain accelerometers and gyroscopes, and researchers have used them to measure movement objectively in cat and dog orthopedic studies. (1,2) But IMUs measure in terms of gravities and angular velocities, which are not clinically relevant units. For clinical trials, clinically relevant units are necessary to show the treatment’s effect in practical ways that matter to dogs and their owners.

This research demonstrates how to use raw IMU data to calculate the number of minutes dogs spend in motion. “Minutes of motion” are clinically relevant, objective measures: Researchers and owners hope less OA pain means more mobility and more time moving.

This research is motivated by long-term orthopedic randomized clinical trials testing the effect of an analgesic for OA pain relief. If the treatment group has, on average, statistically significantly more minutes of motion than the control group, then we would conclude there is a treatment effect. More importantly, we could assess the magnitude of the clinical effect: Did the treatment dogs spend one hour moving more than control dogs per day, or one minute?

My approach was to segment a dog’s IMU data into 7-second-long, non-overlapping windows, then classify each window as stationary or moving. The number of  "moving" windows multiplied by 7 seconds gives the total number of moving seconds for the dog over the trial. From there, dividing the total number of seconds by 60 seconds gives the dog’s minutes of motion during the trial.

The key part of this research was developing a classifier that identifies the widows as stationary or moving. I developed the window classifier using IMU data from a Finnish study of dogs wearing collar IMUs while performing specified stationary tasks (standing, sitting, or lying down) and moving tasks (walking, trotting, playing, and treat searching) for three minutes each. (3,4) 

That data was used to develop decision boundaries that separate stationary-type IMU data from moving-type IMU data. The classifier classifies a window as stationary or moving depending on where the window's data falls among the boundaries.

The simulation results here depend on the Finnish data, but the basic algorithm does not. The algorithm could easily be generalized to different IMUs and species. All that is needed is a  window classifier. Researchers could use any other labeled training data in a supervised classifier or no training data and an unsupervised classifier.

**Bibliography**

1. Scott, R. M., Evans, R., and Conzemius, M. G. (2017). Efficacy of an oral nutraceutical for the treatment of canine osteo arthritis. Veterinary and Comparative Orthopaedics and Traumatology, 30(05), 318-323.

2. Lascelles, B. D. X., Hansen, B. D., Roe, S., DePuy, V., Thomson, A., Pierce, C. C., and Rowinski, E. (2007). Evaluation of client?specific outcome measures and activity monitoring to measure pain relief in cats with osteoarthritis. Journal of Veterinary Internal Medicine, 21(3), 410-416

3. Vehkaoja, Antti; Somppi, Sanni; Törnqvist, Heini; Valldeoriola Cardó, Anna; Kumpulainen, Pekka; Väätäjä, Heli; Majaranta, Päivi; Surakka, Veikko; Kujala, Miiamaaria; Vainio, Outi (2022), “Movement Sensor Dataset for Dog Behavior Classification”, Mendeley Data, V2, doi: 10.17632/vxhx934tbn.2

4. Kumpulainen, P., Valldeoriola Cardó, A., Somppi, S., Törnqvist, H., Väätäjä, H., Majaranta, P., Gizatdinova, Y., Hoog Antink, C., Surakka, V., Kujala, M. V., Vainio, O., and Vehkaoja, A., Dog behavior classification with movement sensors placed on the harness and the collar, Applied Animal Behavior Science, 241 (2021): 105393.





##Results
Over several simulations, ranging from 3 hours to 14 days, the Minutes-of-Motion Algorithm calculated the number of minutes to be between 0.1 percent and 2 percent of a dog's true __moving__ time. 

For example, when, on average, dogs are stationary 50 percent of the time in a three-hour IMU measurement session, (so moving 1.5 hours) the algorithm overestimated movement by a median of 2 minutes.

The accuracy depends upon the kinds of activities that are simulated. There is higher accuracy when the types of movement are more different; for example, standing vs. walking is less accurate than sitting vs. trotting.

The Minutes-of-Motion Algorithm takes only a few minutes to run, even on 45-dog, two-week data. On real data, the algorithm is time efficient. However, generating simulated data takes considerable time. The code below generates simulated as part of the Minutes-of-Motion algorithm.


##Discussion
I measured the algorithm's accuracy seems to depend on the number of true minutes in motion. More motion means better accuracy.

The minutes-in-motion algorithm could distinguish between moving time and stationary time using a classifier trained on previous data, the Finnish dataset. The classifier training data was a subset of the Finnish data, and the simulated clinical trial data was constructed from another subset of the Finnish data. Both datasets had data from the same kind of IMU. Different IMUs may be calibrated differently, so researchers should use care to classify data from one type of IMU with data from another.

But the Finnish data could be used to train classifiers for data from other IMUs. Normalizing the features, as was done here, is some protection against different calibrations. However, if the calibrations are markedly different on a normalized scale, or the IMUs measure differently, then the Finnish data should not be used to train the classifier. Instead, other calibrated data should be used to train a supervised classifier. 


##The Finnish Data
I used a publicly available Finnish dataset for this simulation. The Finnish researchers recorded data at 100 Hz from 45 dogs wearing collar IMUs (ActiGraph GT9X Link, ActiGraph LLC, Florida, USA). There were about 3 minutes of data each for three stationary tasks (standing, sitting, or lying down) and four moving tasks (walking, trotting, playing, and treat searching), for a total of about 21 minutes of data for each dog. Two were measured twice, giving 42 minutes of data for those dogs)

Random snippets of the 21-minute data were combined to simulate multi-day data (see below for a complete description).

The Finnish IMU variables were accelerometer data in three dimensions and gyroscope data in three dimensions ($x,$,$y$,$z$). The gyroscope data adequately discriminated between stationary and moving tasks, so the noisier accelerometer data were not used in this simulation.

[The original data description here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8777071/)



[The data is here](https://data.mendeley.com/datasets/vxhx934tbn/2)



##The Algorithm

In practice, the Minutes-in-Motion algorithm has three steps.
1. Build a classifier that distinguishes moving motion from stationary motion.
1. Apply that classifier to short windows of a dog's IMU data (e.g., 10-second windows) and classify each window as moving or stationary.
1. Add up the number of moving windows and perform the appropriate arithmetic to find the number of minutes in motion. For example, there are $1.21 x 10^8$ time points in two weeks of IMU data (100Hz) from one dog. That's $1.21 x 10^6$ seconds or $1.21 x10^5$ 10-second windows. If they moved in half of the windows, that's $604800$ seconds, or $168.1$ hours of movement in two weeks. 


##The Classifier and Feature Engineering
I used Scilit-Learn's AdaBoost classifier trained on the Finnish data, but other classifiers, including simple decision rules based on eyeballing the data, seemed to work.

I trained the classifier omitting the "treat-search" task. The reason was that the classifier was trained on balanced data (half stationary, half moving). The choice was conservative because the treat-search IMU data had high values and is clearly distinguished from stationary IMU data. On the other hand, walking IMU values sometimes look like standing IMU values. Leaving walking out makes the classifier look better than it is. 

###Features
I tried many summaries, IQR, other percentiles, and the mean, but the classifier used only the median, sometimes not even that. It rarely used the gyro RMS instead of the gyro $z$ dimension. Only gyroscope data was used as the accelerometer data didn't contribute to the classification. Feature engineering was as follows. First, I calculated the root mean square (RMS) of the gyro data and included it as data along with the gyroscope data ($x,$,$y$,$z$). Those four variables were summarized with medians within a task for each dog. That is, a dog's 3-minute task was summarized with four medians corresponding to the axes and the RMS. So, each dog had seven rows of data corresponding to the seven tasks and six columns of data: a column indicating the task (e.g., "trot"), a binary column indicating if that task was stationary or moving, and the four medians. The training set was 315 + 14 (7x45=315. Two dogs were assessed twice for 14) rows and six columns. 

#The Simulation Method
I didn't have real data, so I modified the algorithm slightly for this simulation. Instead of using all the Finnish data to train the classifier, I used a leave-one-out approach.

The steps were,

1. I simulated "clinical trial" long-term data movement data for each dog (see below).
1. For each dog, the classifier was trained on the other 44 dogs. In other words, that particular dog was left out of the classifier training, and it didn't influence its window classifications.
1. Minutes of motion were calculated for each dog and compared to the actual minutes of motion.


Finally, the medians were normalized within dog using a minimax scaler. That way, dogs with different movement styles would be on the same scale.



#The Simulated data

Each simulated dog's long-term data came from its doppelganger in the Finnish set. It was an iterative process. For a given dog, and a given clinical trial length (e.g., 14 days)

1. I randomly selected a task using weighted sampling (see below)
1. I selected a snippet length from 5 seconds to 3 minutes long
1. I sampled a snippet of Finnish IMU data for that dog, task, and length.
1. I repeated that process until the dog had at least the correct 14 days of data. 

Next, the extended data sequence was segmented into 7-second windows, meaning there were 700 data rows in each window. Each window was summarized like the training data, with the medians of the three axes and the RMS. Note that the task was ignored here because that data is missing in an actual clinical trial. 

Part of generating the simulated data was randomly choosing tasks in a way that mimicked real dogs. In the simulations, I generally weighted the dogs, so they had at least 50 percent stationary movement. 

##The Analysis


#Reads the real dogs from the Finnish study

In [1]:
%%time

#this is for local runtime
import pandas as pd
from google.colab import drive


#read the training set
path2 = "your_path_here"
finnish_dog_master = pd.read_csv(path2)



import statistics
import numpy as np
from sklearn.ensemble import AdaBoostClassifier



/opt/anaconda3/lib/python3.9/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


CPU times: user 15.4 s, sys: 1.18 s, total: 16.6 s
Wall time: 16.8 s


##Some Functions

In [2]:

'''
These functions are used to (a) make training data or (b) classify windows
This calculates the median IMU data. '''


def my_median(df_arg):
  
  df_abs = df_arg.abs()
  df_abs_median = df_abs.median(numeric_only=True)**.001
  
  return df_abs_median

#This is the minimax scaler
def my_scaler(z):
  return (z-z.min())/(z.max()-z.min())


'''
Samples a random span from 5sec to 3min  from the dog's entire dataset
that way the data may cover changes in movement but stay contiguous, that is,
not take a single observation from one movement. The problem with that is movement is defined
by a sequence of observations, not just one'''

def sample_movement(dogid,task):
  dog_rows = finnish_dog[(finnish_dog['DogID']==dogid) & (finnish_dog['Task']==task)] #getting all rows with the same, randomly sampled dog
  
  sample_length = np.random.randint(500, (dog_rows.shape[0]-501)) #5 seconds to all but 5 seconds the entire movement (about 3 minutes)
  sample_start = np.random.randint(0, dog_rows.shape[0]-sample_length) # random starting point of a sequence #3000 long is 30 seconds of data. 6000 is one minute of data
  sampled_part =dog_rows.iloc[sample_start:(sample_start+sample_length)]  #sample the dataset
  return sampled_part

This cell prepares the Finnish dataset by removing unused variables and unused tasks.

In [3]:
finnish_dog = finnish_dog_master.copy()
finnish_dog['GRMS'] = np.sqrt(np.square(finnish_dog['GNeck_x']) +	np.square(finnish_dog['GNeck_y']) + np.square(finnish_dog['GNeck_z']))
#remove the back IMU data and the undefined movement data

finnish_dog = finnish_dog[finnish_dog['Task']!='<undefined>'] #remove the undefined movements
finnish_dog = finnish_dog[finnish_dog['Task']!='Task treat-search']

#remove the back IMU data and the undefined movement data
finnish_dog = finnish_dog.drop(['TestNum','t_sec','ABack_x',	'ABack_y',	'ABack_z','ANeck_z', 'GBack_x', 'GBack_y','GBack_z',
                                'Behavior_1','Behavior_2',	'Behavior_3',	'PointEvent',
                                'ANeck_x', 'ANeck_y'], axis = 1)



 

This cell creates the features

In [4]:
#summarize within dog and task
labeled_dog_median = finnish_dog.groupby(['DogID','Task']).apply(my_median)

#label moving vs unmoving
mask = ((finnish_dog['Task'] == 'Task lie down') | (finnish_dog['Task'] == 'Task sit') |(finnish_dog['Task'] == 'Task stand'))
finnish_dog['moving'] = np.where(mask, 0, 1)

'''
The line below is the labels for the training data. It's a hack--I'm taking median of 
the moving variable within a task, but within a task moving is all either 0 or 1. 
'''
y = finnish_dog.groupby(['DogID','Task']).median()['moving'].astype(int)



#scale within dog by column
normalized_X = labeled_dog_median.groupby('DogID').apply(my_scaler)



#Minutes of motion function


In [5]:

def min_of_motion(dog_id,window_size,df): #df is the dataframe of raw IMU data

  #remove the target dog from the training set.
  X_train = normalized_X.drop(index=dog_id)
  y_train = y.drop(index=dog_id)

 # #train the classifier
  model = AdaBoostClassifier(n_estimators=100, learning_rate=1 ,random_state=5)
  model.fit(X_train, y_train)


  unlabeled_dog = df[df['DogID']==dog_id]   #keep only the target dog's data

  #calc the number of windows, but losing a few seconds on the end if there isn't a complete window 
  num_windows = int(unlabeled_dog.shape[0]/(window_size*100)) #the 100 coverts to Hz

  unlabeled_dog = unlabeled_dog.iloc[:(num_windows*100*window_size)] #

  
  #this makes a list of integers that is the largest divisor of n seconds the length of simulated dog 
  #there will be some of the simulated dog that doesn't fit
  values = [x for x in range(0,num_windows)] 


  #repeats the window many times to make the windows.
  # and sorts the windows so all the 1's are first, 2's are second and so on
  values_sorted = np.sort(np.resize(values,unlabeled_dog.shape[0]))  


  #make the new columns of windows. This way avoid a copy over problem
  unlabeled_dog.insert(2, "windows", values_sorted, True)


  #calculate the true minutes moving from the test dataset. This is used later
  simul_true_minutes = unlabeled_dog['moving'].sum()/100/60
  
  #drop unused columns. May have to change this if I import a file that doesn't have "moving"
  unlabeled_dog.drop(labels=['DogID','moving','Task'], axis=1, inplace=True) #this drops the old row numbers 

  #Already selected on dog_id
  X_sim = unlabeled_dog.groupby(['windows']).apply(my_median)
  X_sim = X_sim.drop('windows',axis=1)

  
  #normalize
  X_sim = (X_sim-X_sim.min())/(X_sim.max()-X_sim.min())

  
  y_pred_simul = model.predict(X_sim) #predictions from the classifter for whole simulated dog


  simul_predicted_minutes = y_pred_simul.sum()*window_size/60   #the /60 converts from seconds to minutes
  
  return [simul_predicted_minutes, simul_true_minutes,y_pred_simul.shape[0]*window_size/60]

##This cell generates the simulated data

In [ ]:
%%time

#define the output dataframe. It will hold all the dogs' minute counts
out_df = pd.DataFrame(columns=['Dogid','pred_min','true_min','sample_length_min'])


#60480000  seven days 100hz
#8640000 one day 100Hz
# 386400 about an hour , so, 3*386400 is three hours

NUMBER_OBS_HZ = 3*386400    #change this number to get longer or shorter clinical trial 

WINDOW_SIZE = 7   #the widow size in seconds. Change here. 

TASK_LIST = ['Task walk', 'Task sit', 'Task trot', 'Task lie down', 'Task play', 'Task stand', 'Task treat-search']

TASK_PROBABILITIES = [0.0,0.0,0.0,0.5,0.5,0.0,0.0]  #ordering matches TASK_LIST. The numbers add to 1.

'''
list of dogs to simulate. For longer simulations use fewer dogs to save time. For example, DOG_ID_LIST =[16, 18]. DOG_ID_LIST = finnish_dog['DogID'].unique() 
is all the dogs. Note that the dog IDs start at 16 and skip some numbers. 
'''
DOG_ID_LIST = finnish_dog['DogID'].unique() 


for j in DOG_ID_LIST: #loop over the dogs, leaving one out at a time
  print(j)  #print the current dog ID just to keep track of how long the loop is taking
  
  
  #An empty datframe that will temporarily hold a single dog's simulated data, which is IMU data at 100 Hz
  simul_dog_raw = pd.DataFrame()

  while (simul_dog_raw.shape[0])<NUMBER_OBS_HZ:
      draw_task = np.random.choice(a=TASK_LIST,size=1,replace=True, p=TASK_PROBABILITIES)    #draw random tasks

      simul_dog_raw = pd.concat([simul_dog_raw,sample_movement(j,draw_task[0])])      #sample the specificied task randomly and concat to make long
  
  
  #identify task as moving vs stationary. This is needed here for the time calculations at the end
  mask2 = ((simul_dog_raw['Task'] == 'Task lie down') | (simul_dog_raw['Task'] == 'Task sit') |(simul_dog_raw['Task'] == 'Task stand'))
  simul_dog_raw['moving'] = np.where(mask2, 0, 1)
  

  
  simul_dog_minutes = min_of_motion(j,WINDOW_SIZE,simul_dog_raw) #Find the number of Minutes of Motion for dog j

  out_df = pd.concat([out_df,
                     pd.DataFrame.from_dict([{"Dogid" :j,
                                    "pred_min": simul_dog_minutes[0],
                                    "true_min": simul_dog_minutes[1],
                                    "sample_length_min":simul_dog_minutes[2] }])])
  


#write the output file

In [7]:

#path = "your_path"
#out_df.to_csv(path)

#Statistical Analysis

In [8]:
#Glance at the output file of predicted and true active minutes, and total time
out_df.head()

,Dogid,pred_min,true_min,sample_length_min
0,16,125.533333,113.177333,197.516667
0,18,98.7,102.1065,193.666667
0,19,64.05,86.645,193.316667
0,20,110.716667,112.385333,195.883333
0,21,91.0,91.4985,194.25


In [9]:
#Summaries of active minutes, over dog
out_df.astype(float).describe()

,Dogid,pred_min,true_min,sample_length_min
count,45.000000,45.000000,45.000000,45.000000
mean,45.400000,99.495926,99.254100,194.877407
std,17.411334,17.436483,13.616988,1.293008
min,16.000000,64.050000,64.325667,193.200000
25%,29.000000,89.950000,91.498500,193.900000
50%,47.000000,102.316667,99.546667,194.483333
75%,59.000000,110.600000,106.849000,195.650000
max,74.000000,136.966667,134.652333,198.100000


In [10]:
#Summaries of the raw difference in active minutes, predicted - true
pd.DataFrame((out_df['pred_min']-out_df['true_min'])).astype(float).describe()

,0
count,45.000000
mean,0.241826
std,10.410718
min,-23.011167
25%,-3.327167
50%,-0.160333
75%,2.475167
max,39.769167


In [11]:
#Summaries of the proportion: of difference in minutes divided by the true number of active minutes
pd.DataFrame(((out_df['pred_min']-out_df['true_min'])/out_df['true_min'])).astype(float).describe()

,0
count,45.000000
mean,0.001944
std,0.113411
min,-0.260777
25%,-0.032685
50%,-0.001346
75%,0.025384
max,0.409158


In [12]:
#Summaries of the proportion: of difference in minutes divided by the TOTAL number of minutes, active and stationary
pd.DataFrame((out_df['pred_min']-out_df['true_min'])/out_df['sample_length_min']).astype(float).describe()

,0
count,45.000000
mean,0.001206
std,0.053505
min,-0.117614
25%,-0.017016
50%,-0.000816
75%,0.012696
max,0.204978


Pairwise t-test to compare predicted and true minutes. It's really the wrong test. An equivalence test would be better. 

In [13]:
import scipy.stats as stats

stats.ttest_rel(out_df['pred_min'], out_df['true_min'])

Ttest_relResult(statistic=0.15582188153314472, pvalue=0.8768861297819946)

Wilcoxon signed-rank test to compare predicted and true minutes. It's really the wrong test. An equivalence test would be better. 

In [14]:
from scipy.stats import wilcoxon
res = wilcoxon((out_df['pred_min']-out_df['true_min']),alternative='greater')
res.statistic, res.pvalue

(490.0, 0.6218745194189702)

The time length of the clinical trial

In [15]:
print('minutes:' , NUMBER_OBS_HZ/100/60)
print('hours:' ,NUMBER_OBS_HZ/100/60/60)
print('days:' ,NUMBER_OBS_HZ/100/60/60/24)

minutes: 193.2
hours: 3.2199999999999998
days: 0.13416666666666666
